# Turing patterns and your own automata (16 points)

Animal skin patterns are a beautiful and intriguing example of pattern formation in biology. They were studied by famous English mathematician Alan Turing, who, in 1952, published a paper titled *The Chemical Basis of Morphogenesis*. In his model, Turing described interactions between two homogeneneously distributed substances, that produce stable patterns. His model used partial differential equations. During this classes you'll implement a simpler, CA version of this model developed by [David Young](https://www.sciencedirect.com/science/article/abs/pii/0025556484900609).
![image](https://upload.wikimedia.org/wikipedia/commons/a/a1/TuringPattern.PNG)
<center>(source: <a href="https://upload.wikimedia.org/wikipedia/commons/a/a1/TuringPattern.PNG">wikimedia.org</a>)</center>

## David Young model

##### Grid
Two-dimensional rectangular grid.

##### Cells
Binary cells: `0` (inactive/passive) or `1` (active).

##### Neighbourhood
This model uses two Moore neighbourhoods. The first, with radius $R_a$, is used for determining the number of active cells (*short-range activation*). The second one, with greater radius $R_i$ is used for counting the amount of inactive/passive cells (*long-range inhibition*). $R_a$ and $R_i$ ($R_a<R_i$), as well as $w_a$ and $w_i$, are the model's parameters. We're considering only odd $R_a$ and $R_i$ values.

##### Model
Mathematically speaking, the model can be described as (Hiroki Sayama, *Introduction to the Modeling and Analysis of Complex Systems*):
* Short-range neighbourhood:</br>$$N_a = \left\{ x'\left| |x'|\right. \leq R_a \right\}$$</br>
* Long-range neighbourhood:</br> $$N_i = \left\{ x'\left| |x'|\right. \leq R_i \right\}$$ </br>
* Transition function:</br>$$a_t(x) = w_a\sum_{x' \in N_a}s_t(x+x')-w_i\sum_{x' \in N_i}s_t(x+x')$$ $s_t$ is a mathematical function that maps location to state, the sum is simply a sum of neighbours values</br> $$s_{t+1}(x) = \left\{ \begin{array}{ll}1 & \text{if } a_t(x) > 0\\0 & \text{otherwise}\end{array}\right.$$


I assume thet the is a mistake in above description and there should be:

$$a_t(x) = w_a\sum_{x' \in N_a}s_t(x)- s_t(x')-w_i\sum_{x' \in N_i}s_t(x)-s_t(x')$$ 

## Python implementation of David Young's model (4 points)
Fill in the gaps in the following code.

In [66]:
import numpy as np
import itertools
import pygame

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    K_SPACE,
    KEYDOWN,
    QUIT,
)

In [67]:
# Grid size
rows = 80
cols = 80

In [68]:
class Model:
    def __init__(self, rows=80, cols=80, Ra=3, Ri=5, wa=1, wi=0.2, p1=0.01):
        self.grid = np.random.choice([0, 1], size=(rows, cols), p=[1 - p1, p1])
        self.Ra = Ra
        self.Ri = Ri
        self.wa = wa
        self.wi = wi
    
    def neighbor_sum(self, x, y, R):
        sum_neighbors = 0
        for dx in range(x-R, x+R+1):
            for dy in range(y-R, y+R+1):
                if x == dx and y == dy: continue
                if (dx - x) ** 2 + (dy - y) ** 2 <= R ** 2:
                    sum_neighbors += self.grid[x, y] - self.grid[dx, dy]
        return sum_neighbors
    
    def update(self):
        new_grid = np.zeros_like(self.grid)
        rows, cols = self.grid.shape
        for x in range(self.Ri, rows-self.Ri):
            for y in range(self.Ri, cols-self.Ri):
                short_range_sum = self.neighbor_sum(x, y, self.Ra)
                long_range_sum = self.neighbor_sum(x, y, self.Ri)
                a_t_x = self.wa * short_range_sum - self.wi * long_range_sum
                new_grid[x, y] = 1 if a_t_x > 0 else 0
        self.grid = new_grid

    def drawGrid(self, screen,w_width, w_height):
        black = (0,0,0)
        white = (255,255,255)
        rows, cols = self.grid.shape
        blockSize = (min(w_width, w_height)-max(rows, cols))/max(rows, cols)
        for x in range(self.Ri, rows-self.Ri):
            for y in range(self.Ri, cols-self.Ri):
                pos_x = (blockSize+1)*x
                pos_y = (blockSize+1)*y
                rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
                if self.grid[x][y] == 1:
                    pygame.draw.rect(screen, black, rect, 0)
                else:
                    pygame.draw.rect(screen, white, rect, 0)    
        pygame.display.flip()

In [69]:
pygame.init()
w_width = 800
w_height = 800
# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])
model = Model(rows,cols, Ra=3, Ri=5, wa=1, wi=0.2, p1=0.01)

# Set background
screen.fill((128, 128, 128))

model.drawGrid(screen, w_width, w_height)
    
running = True

time_delay = 500
timer_event = pygame.USEREVENT + 1
pygame.time.set_timer(timer_event, time_delay )

while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                model.update()
                model.drawGrid(screen, w_width, w_height)
        if event.type == timer_event:
            model.update()
            model.drawGrid(screen, w_width, w_height)


pygame.quit()

## Analysis (5 points)
Check different values of $w_a, w_i, R_a, R_b$. Find 5 visually different stable or oscilating patterns. Write down the parameteres and present results. Two sets of parameteres are considered to be different if at least two of the parameters are **substantially** different.

In [70]:
def simulate(model):    
    pygame.init()
    w_width = 800
    w_height = 800
    # Set up the drawing window, adjust the size
    screen = pygame.display.set_mode([w_width, w_height])

    # Set background
    screen.fill((128, 128, 128))

    model.drawGrid(screen, w_width, w_height)
        
    running = True

    time_delay = 500
    timer_event = pygame.USEREVENT + 1
    pygame.time.set_timer(timer_event, time_delay )

    while running:
        for event in pygame.event.get():   
            if event.type == QUIT:
                running = False
            
            if event.type == KEYDOWN:
                if event.key == K_RIGHT:
                    model.update()
                    model.drawGrid(screen, w_width, w_height)
            if event.type == timer_event:
                model.update()
                model.drawGrid(screen, w_width, w_height)


    pygame.quit()

In [71]:
simulate(Model(Ra=3, Ri=5, wa=1, wi=0.2))

In [72]:
simulate(Model(Ra=3, Ri=4, wa=0.99, wi=0.1))

In [73]:
simulate(Model(Ra=3, Ri=6, wa=1, wi=0.14))

In [74]:
simulate(Model(Ra=3, Ri=3, wa=2, wi=0.1))

In [75]:
simulate(Model(Ra=3, Ri=9, wa=2, wi=0.1))

## Improvements (3 points)
In order to get this points you have to complete all of the previous tasks. Add widgets for parameters changing (e.g. slider or text field)


In [76]:
class Slider():
    WHITE = (255, 255, 255)
    GRAY = (200, 200, 200)
    DARK_GRAY = (150, 150, 150)
    BLUE = (0, 0, 255)
    with_of_text_filed = 150

    def __init__(self, x, y, width, name, value, min_val, max_val):
        self.x = x
        self.y = y
        self.width = width - self.with_of_text_filed 
        self.name = name
        self.value = value
        self.min_val = min_val
        self.max_val = max_val
        self.dragging = False
        self.calculate_position()

    def get_value(self):
        return self.value
    
    def calculate_position(self):
        self.position_x = self.x + (((self.value - self.min_val) / (self.max_val - self.min_val)) * self.width)

    def calculate_value(self, new_position):
        new_value = max(self.min_val, min(self.max_val, ((new_position - self.x) / self.width) * (self.max_val - self.min_val)))
        if (self.max_val - self.min_val) < 1:
            self.value = round(new_value, 3)
        elif (self.max_val - self.min_val) < 10:
            self.value = round(new_value, 2)
        elif (self.max_val - self.min_val) < 300:
            self.value = round(new_value)
        else:
            self.value = round(new_value, -2)

    def draw(self, screen):
        pygame.draw.rect(screen, self.WHITE, (self.x-15, self.y-15, self.width + self.with_of_text_filed, 30))
        pygame.draw.rect(screen, self.GRAY, (self.x-2, self.y-2, self.width, 4))
        pygame.draw.circle(screen, self.BLUE, (self.position_x, self.y), 15)
        text = pygame.font.Font(None, 30).render(f"{self.name}: {self.value}", True, self.DARK_GRAY)
        screen.blit(text, (self.x + self.width + 15, self.y - 10))

    def handle(self, event, screen):
        if not event: pass
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouse_x, mouse_y = event.pos
            if abs(mouse_x - self.position_x) <= 15 and abs(mouse_y - self.y) <= 15:
                self.dragging = True
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
        elif event.type == pygame.MOUSEMOTION:
            if self.dragging:
                mouse_x = event.pos[0]
                self.calculate_value(mouse_x)
                self.calculate_position()

        self.draw(screen)

In [87]:
class Imp_model(Model):
    def __init__(self, rows=80, cols=80, Ra=3, Ri=5, wa=1, wi=0.2, p1=0.01):
        super().__init__(rows, cols, Ra, Ri, wa, wi, p1)
        self.sliders_width = 450
        self.w_width = (rows - 2*self.Ri)*10 + self.sliders_width + 10
        self.w_height = (rows - 2*self.Ri)*10
        self.instruction = (
            "INSTRUCTIONS:\n"
            "Press RIGHTARROW to update\n"
            "Press SPACE for loop\n"
            "Use sliders to change parameters\n"
            "(when in loop - lowers performance)"
        )
        self.init_sliders()
        
        
    def simulate(self):
        pygame.init()
        self.screen = pygame.display.set_mode([self.w_width, self.w_height])
        self.screen.fill((128, 128, 128))
        self.drawGrid()
        self.handle_sliders(event=None)
        self.draw_instructions()
            
        running = True
        loop = False
        while running:
            for event in pygame.event.get():   
                self.handle_sliders(event)
                if event.type == QUIT:
                    running = False
                
                if event.type == KEYDOWN:
                    if event.key == K_RIGHT:
                        self.update()
                        self.drawGrid()
                    if event.key == K_SPACE:
                        loop = not loop
            if loop:
                pygame.time.wait(100)
                self.update()
                self.drawGrid()
            
            pygame.display.flip()

        pygame.quit()

    def draw_instructions(self):
        if not self.instruction: return
        x_instr, y_instr = self.w_width - self.sliders_width, self.w_height - 30
        for i, line in enumerate(reversed(self.instruction.split("\n"))):
            text = pygame.font.Font(None, 30).render(line, True, (0, 0, 0))
            self.screen.blit(text, (x_instr, y_instr - 30 * i))

    def drawGrid(self):
        black = (0,0,0)
        white = (255,255,255)
        rows, cols = self.grid.shape
        blockSize = int(self.w_height/(rows - self.Ri*2)) - 1 # some place for border
        for idx_x, x in enumerate(range(self.Ri, rows-self.Ri)):
            for idx_y, y in enumerate(range(self.Ri, cols-self.Ri)):
                rect = pygame.Rect(idx_x*(blockSize+1), idx_y*(blockSize+1), blockSize, blockSize)
                if self.grid[x][y] == 1:
                    pygame.draw.rect(self.screen, black, rect, 0)
                else:
                    pygame.draw.rect(self.screen, white, rect, 0)
        pygame.display.flip()

    def init_sliders(self):
        x_slider, y_slider = self.w_width - self.sliders_width + 10, 15
        self.wa_slider = Slider(x=x_slider, y=y_slider, width=self.sliders_width,
                                name="wa", value=self.wa, min_val=0, max_val=2)
        self.wi_slider = Slider(x=x_slider, y=y_slider + 30, width=self.sliders_width,
                                name="wi", value=self.wi, min_val=0, max_val=2)

    def handle_sliders(self, event):
        self.wa_slider.handle(event, self.screen)
        self.wa = self.wa_slider.get_value()
        self.wi_slider.handle(event, self.screen)
        self.wi = self.wi_slider.get_value()

In [88]:
Imp_model().simulate()

# Your cellular automata (4 points)

In order to get this points you have to complete all of the previous tasks.
Find another example of CA based model, descirbe the rules and implement the automata

## Forest-fire model

[Wikipedia](https://en.wikipedia.org/wiki/Forest-fire_model)

This model operates on a grid where each cell represents a small area of the forest that can be in one of three states:

*    Tree (1): The cell contains a tree.
*    Burning (2): The cell contains a burning tree.
*    Empty (0): The cell is empty, either because it was previously burned or no tree was there.

The Forest Fire Model follows these basic rules:

*    If a cell contains a burning tree, it turns into an empty cell in the next time step, as the fire consumes the tree.
*    If a cell contains a tree and at least one of its neighboring cells is burning, it will catch fire and burn in the next time step.
*    If a cell is empty, it remains empty unless new trees grow in the model.

In [91]:
class ForestFire(Imp_model):
    def __init__(self, p=0.005, f=0.001):
        self.p = p
        self.f = f
        super().__init__()
        self.grid = np.random.choice([0, 1, 2], size=(self.grid.shape), p=[0.199, 0.8, 0.001])
        self.w_width = 800 + self.sliders_width + 10
        self.w_height = 800
        self.instruction = (
            "FOREST FIRE CA\n"
            "p - probability of a new tree growing\n"
            "f - probability of spontaneous fire ignition\n\n" 
            + self.instruction
        )        
        self.init_sliders()
    
    def get_neighbors(self, x, y):
        neighbors = []
        rows, cols = self.grid.shape
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # Up, down, left, right
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols:
                neighbors.append((nx, ny))
        return neighbors

    #Override
    def update(self):
        new_grid = np.zeros_like(self.grid)
        rows, cols = self.grid.shape
        for x in range(rows):
            for y in range(cols):
                if self.grid[x][y] == 2:
                    new_grid[x, y] = 0
                elif self.grid[x][y] == 1:
                    if any(self.grid[nx, ny] == 2 for nx, ny in self.get_neighbors(x, y)) or \
                            np.random.rand() < self.f:
                        new_grid[x, y] = 2
                    else:
                        new_grid[x, y] = 1
                else:
                    if np.random.rand() < self.p:
                        new_grid[x, y] = 1
        self.grid = new_grid

    #Override
    def drawGrid(self):
        white = (255, 255, 255)
        green = (0, 125, 0)
        red = (255, 0, 0)
        rows, cols = self.grid.shape
        blockSize = int(self.w_height/rows)
        for x in range(rows):
            for y in range(cols):
                rect = pygame.Rect(x*blockSize, y*blockSize, blockSize, blockSize)
                if self.grid[x][y] == 1:
                    pygame.draw.rect(self.screen, green, rect, 0)
                elif self.grid[x][y] == 2:
                    pygame.draw.rect(self.screen, red, rect, 0)
                else:
                    pygame.draw.rect(self.screen, white, rect, 0)
        pygame.display.flip()
    
    #Override
    def init_sliders(self):
        x_slider, y_slider = self.w_width - self.sliders_width + 10, 15
        self.p_slider = Slider(x=x_slider, y=y_slider, width=self.sliders_width,
                                name="p", value=self.p, min_val=0, max_val=0.1)
        self.f_slider = Slider(x=x_slider, y=y_slider + 30, width=self.sliders_width,
                                name="f", value=self.f, min_val=0, max_val=0.05)
        
    #Override
    def handle_sliders(self, event):
        self.p_slider.handle(event, self.screen)
        self.p = self.p_slider.get_value()
        self.f_slider.handle(event, self.screen)
        self.f = self.f_slider.get_value()

In [92]:
ForestFire().simulate()

Version that reduces caching frequency, lowering the likelihood from 100% to custom

In [ ]:
class ForestFire85(ForestFire):
    def __init__(self,p,f, fire_spred):
        super().__init__(p,f)
        self.fs = fire_spred
        self.instruction = f"FIRE SPREAD: {fire_spred*100}%\n" + self.instruction

    #Override
    def update(self):
        new_grid = np.zeros_like(self.grid)
        rows, cols = self.grid.shape
        for x in range(rows):
            for y in range(cols):
                if self.grid[x][y] == 2:
                    new_grid[x, y] = 0
                elif self.grid[x][y] == 1:
                    if (any(self.grid[nx, ny] == 2 for nx, ny in self.get_neighbors(x, y)) and \
                            np.random.rand() < self.fs) or np.random.rand() < self.f: #key change
                        new_grid[x, y] = 2
                    else:
                        new_grid[x, y] = 1
                else:
                    if np.random.rand() < self.p:
                        new_grid[x, y] = 1
        self.grid = new_grid
    

In [97]:
ForestFire85(0.005, 0.001, fire_spred=0.60).simulate()